# Homework IV - Aprendizagem

**Afonso da Conceição Ribeiro**, 102763
<br>
**Miguel Gomes Marques Pessanha de Almeida**, 103493

## I. Pen and Paper [11v]

**Given the following observations,**
$
\left\{
\begin{bmatrix} 1 \\  0.6 \\  0.1 \end{bmatrix},
\begin{bmatrix} 0 \\ -0.4 \\  0.8 \end{bmatrix},
\begin{bmatrix} 0 \\  0.2 \\  0.5 \end{bmatrix},
\begin{bmatrix} 1 \\  0.4 \\ -0.1 \end{bmatrix}
\right\}
$
**. <br> Consider a Bayesian clustering that assumes {y<sub>1</sub>} ⫫ {y<sub>2</sub>, y<sub>3</sub>}, two clusters following a Bernoulli distribution on y<sub>1</sub> (p<sub>1</sub> and p<sub>2</sub>), a multivariate Gaussian on {y<sub>2</sub>, y<sub>3</sub>} (N<sub>1</sub> and N<sub>2</sub>), and the following initial mixture:**
$$
π_1 = 0.5 \;, \quad π_2 = 0.5
$$
$$
p_1 = P(y_1 = 1) = 0.3 \;, \quad p_2 = P(y_1 = 1) = 0.7
$$
$$
N_1 \left(
	\textbf{μ}_1 = \begin{bmatrix} 1 \\ 1 \end{bmatrix}, \;
	\textbf{Σ}_1 = \begin{bmatrix} 2 & 0.5 \\ 0.5 & 2 \end{bmatrix}
\right), \quad
N_2 \left(
	\textbf{μ}_2 = \begin{bmatrix} 0 \\ 0 \end{bmatrix}, \;
	\textbf{Σ}_2 = \begin{bmatrix} 1.5 & 1 \\ 1 & 1.5 \end{bmatrix}
\right).
$$

In [ ]:
import numpy as np
from scipy.stats import norm

### 1) [6v]

**Perform one epoch of the EM clustering algorithm and determine the new parameters. <br>
Hint: we suggest you to use `numpy` and `scipy`, however disclose the intermediary results step by step.**

In [ ]:
x1 = np.array([1, 0.6, 0.1])
x2 = np.array([0, -0.4, 0.8])
x3 = np.array([0, 0.2, 0.5])
x4 = np.array([1, 0.4, -0.1])
π1 = 0.5
π2 = 0.5
p1 = 0.3
p2 = 0.7
μ1 = np.array([1, 1])
Σ1 = np.array([[2, 0.5], [0.5, 2]])
μ2 = np.array([0, 0])
Σ2 = np.array([[1.5, 1], [1, 1.5]])

def gaussian(x, μ, Σ):
    return (1 / np.sqrt(np.linalg.det(2 * np.pi * Σ))) * np.exp(-0.5 * np.dot(np.dot((x - μ).T, np.linalg.inv(Σ)), (x - μ)))

X = np.array([x1, x2, x3, x4])
π = np.array([π1, π2])
p = np.array([p1, p2])
μ = np.array([μ1, μ2])
Σ = np.array([Σ1, Σ2])


# E-Step

print("Gaussians:")
gaussians = np.zeros((2, 4))
for k in range(1, 3):
    for i in range(1, 5):
        gaussians[k-1][i-1] = gaussian(X[i-1][1:], μ[k-1], Σ[k-1])
        formatted_pdf = "{:.8f}".format(gaussians[k-1][i-1])
        print("N(x{} | μ{}, Σ{}) = {}".format(i, k, k, formatted_pdf))

# print("\nPosteriors:")
posteriors = np.zeros((2, 4))
for k in range(1, 3):
    for i in range(1, 5):
        posteriors[k-1, i-1] = (p[k-1] * gaussians[k-1, i-1] * π[k-1]) / (factors[0, i-1] + factors[1, i-1])
        # print("P(c{} | x{}) = {:.8f}".format(k, i, posteriors[k-1, i-1]))


# M-Step

# N_k
N1 = np.sum(posteriors[0])
N2 = np.sum(posteriors[1])
# print("N1 = {:.8f}".format(N1))
# print("N2 = {:.8f}".format(N2))

# π_k
π1_new = N1 / (N1 + N2)
π2_new = N2 / (N1 + N2)
print("\nπ1 = {:.8f}".format(π1_new))
print("π2 = {:.8f}".format(π2_new))

# p_k
p1_new = np.sum(posteriors[0] * X[:, 0]) / N1
p2_new = np.sum(posteriors[1] * X[:, 0]) / N2
print("\np1 = {:.8f}".format(p1_new))
print("p2 = {:.8f}".format(p2_new))

# μ_k
μ1_new = np.zeros(2)
μ2_new = np.zeros(2)
for i in range(1, 5):
    μ1_new += posteriors[0, i-1] * X[i-1][1:]
    μ2_new += posteriors[1, i-1] * X[i-1][1:]
μ1_new /= N1
μ2_new /= N2
print("\nμ1 = {}".format(μ1_new))
print("μ2 = {}".format(μ2_new))

# Σ_k
Σ1_new = np.zeros((2, 2))
Σ2_new = np.zeros((2, 2))
for i in range(1, 5):
    Σ1_new += posteriors[0, i-1] * np.outer(X[i-1][1:] - μ1_new, X[i-1][1:] - μ1_new)
    Σ2_new += posteriors[1, i-1] * np.outer(X[i-1][1:] - μ2_new, X[i-1][1:] - μ2_new)
Σ1_new /= N1
Σ2_new /= N2
print("\nΣ1 = {}".format(Σ1_new))
print("Σ2 = {}".format(Σ2_new))

Gaussians:
N(x1 | μ1, Σ1) = 0.06657530
N(x2 | μ1, Σ1) = 0.05004889
N(x3 | μ1, Σ1) = 0.06837452
N(x4 | μ1, Σ1) = 0.05904699
N(x1 | μ2, Σ2) = 0.11961837
N(x2 | μ2, Σ2) = 0.06819058
N(x3 | μ2, Σ2) = 0.12958103
N(x4 | μ2, Σ2) = 0.12450009

π1 = 0.19630768
π2 = 0.80369232

p1 = 0.46039185
p2 = 0.50967458

μ1 = [0.15829218 0.36423748]
μ2 = [0.21018744 0.31541596]

Σ1 = [[ 0.15581144 -0.12557173]
 [-0.12557173  0.12568249]]
Σ2 = [[ 0.13560926 -0.11192041]
 [-0.11192041  0.12047709]]


##### E-Step

$$ \text{Prior:} \; P(c_k) = π_k $$

$$
\text{Likelihood:} \;
    P(\textbf{x}_i \; | \; c_k) = p_k \cdot
        N \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} \; | \; \textbf{μ}_k, \textbf{Σ}_k \right)
$$

$$
\text{Normalized posterior:} \;
    γ_{k,i} = P(c_k \; | \; \textbf{x}_i) = \frac{P(\textbf{x}_i \; | \; c_k) \cdot P(c_k)}
        {\displaystyle\sum_{j = 1}^n \; P(\textbf{x}_i \; | \; c_j) \cdot P(c_j)}
        =
        \frac{p_k \cdot
        N \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} \; | \; \textbf{μ}_k, \textbf{Σ}_k \right) \cdot π_k}
        {\displaystyle\sum_{j = 1}^n \; p_j \cdot
        N \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} \; | \; \textbf{μ}_j, \textbf{Σ}_j \right) \cdot π_j}
$$

$$ γ_{1,1} = P(c_1 \; | \; \textbf{x}_1) = \frac{0.3 \cdot 0.06657530 \cdot 0.5}{0.3 \cdot 0.06657530 \cdot 0.5 + 0.7 \cdot 0.11961837 \cdot 0.5} = 0.19258959 $$
$$ γ_{1,2} = P(c_1 \; | \; \textbf{x}_2) = \frac{0.3 \cdot 0.05004889 \cdot 0.5}{0.3 \cdot 0.05004889 \cdot 0.5 + 0.7 \cdot 0.06819058 \cdot 0.5} = 0.23928490 $$
$$ γ_{1,3} = P(c_1 \; | \; \textbf{x}_3) = \frac{0.3 \cdot 0.06837452 \cdot 0.5}{0.3 \cdot 0.06837452 \cdot 0.5 + 0.7 \cdot 0.12958103 \cdot 0.5} = 0.18443199 $$
$$ γ_{1,4} = P(c_1 \; | \; \textbf{x}_4) = \frac{0.3 \cdot 0.05904699 \cdot 0.5}{0.3 \cdot 0.05904699 \cdot 0.5 + 0.7 \cdot 0.12450009 \cdot 0.5} = 0.16892423 $$
$$ γ_{2,1} = P(c_2 \; | \; \textbf{x}_1) = \frac{0.7 \cdot 0.11961837 \cdot 0.5}{0.3 \cdot 0.06657530 \cdot 0.5 + 0.7 \cdot 0.11961837 \cdot 0.5} = 0.80741041 $$
$$ γ_{2,2} = P(c_2 \; | \; \textbf{x}_2) = \frac{0.7 \cdot 0.06819058 \cdot 0.5}{0.3 \cdot 0.05004889 \cdot 0.5 + 0.7 \cdot 0.06819058 \cdot 0.5} = 0.76071510 $$
$$ γ_{2,3} = P(c_2 \; | \; \textbf{x}_3) = \frac{0.7 \cdot 0.12958103 \cdot 0.5}{0.3 \cdot 0.06837452 \cdot 0.5 + 0.7 \cdot 0.12958103 \cdot 0.5} = 0.81556801 $$
$$ γ_{2,4} = P(c_2 \; | \; \textbf{x}_4) = \frac{0.7 \cdot 0.12450009 \cdot 0.5}{0.3 \cdot 0.05904699 \cdot 0.5 + 0.7 \cdot 0.12450009 \cdot 0.5} = 0.83107577 $$

##### M-Step

$$ N_k = \sum_{i = 1}^n γ_{k,i} $$

$$ π_k = \frac{N_k}{\displaystyle\sum_{j = 1}^2 N_j} $$

$$ p_k = \frac{1}{N_k} \; \sum_{i = 1}^n γ_{k,i} \cdot x_{i1} $$

$$ \textbf{μ}_k = \frac{1}{N_k} \; \sum_{i = 1}^n γ_{k,i} \cdot \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} $$

$$ \textbf{Σ}_k = \frac{1}{N_k} \; \sum_{i = 1}^n γ_{k,i} \cdot \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} - \textbf{μ}_k \right) \cdot \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} - \textbf{μ}_k \right)^T

$$ N_1 = 0.19258959 + 0.23928490 + 0.18443199 + 0.16892423 = 0.78523072 $$
$$ N_2 = 0.80741041 + 0.76071510 + 0.81556801 + 0.83107577 = 3.21476928 $$

$$ π_1 = \frac{0.78523072}{0.78523072 + 3.21476928} = 0.19630768 $$
$$ π_2 = \frac{3.21476928}{0.78523072 + 3.21476928} = 0.80369232 $$

$$ p_1 = \frac{1}{0.78523072} \cdot
    (0.19258959 \cdot 1 + 0.23928490 \cdot 0 + 0.18443199 \cdot 0 + 0. 16892423 \cdot 1)
    = 0.46039185 $$

$$ p_2 = \frac{1}{3.21476928} \cdot
    (0.80741041 \cdot 1 + 0.76071510 \cdot 0 + 0.81556801 \cdot 0 + 0.83107577 \cdot 1)
    = 0.50967458 $$

$$ \textbf{μ}_1 = \frac{1}{0.78523072} \cdot \left( \begin{align*}
        0.19258959 \cdot \begin{bmatrix}  0.6 \\  0.1 \end{bmatrix} +
        0.23928490 \cdot \begin{bmatrix} -0.4 \\  0.8 \end{bmatrix} + \\
        0.18443199 \cdot \begin{bmatrix}  0.2 \\  0.5 \end{bmatrix} +
        0.16892423 \cdot \begin{bmatrix}  0.4 \\ -0.1 \end{bmatrix} \end{align*}
    \right)
    =
    \begin{bmatrix} 0.15829218 \\ 0.36423748 \end{bmatrix} $$

$$ \textbf{μ}_2 = \frac{1}{3.21476928} \cdot \left( \begin{align*}
        0.80741041 \cdot \begin{bmatrix}  0.6 \\  0.1 \end{bmatrix} +
        0.76071510 \cdot \begin{bmatrix} -0.4 \\  0.8 \end{bmatrix} + \\
        0.81556801 \cdot \begin{bmatrix}  0.2 \\  0.5 \end{bmatrix} +
        0.83107577 \cdot \begin{bmatrix}  0.4 \\ -0.1 \end{bmatrix} \end{align*}
    \right)
    =
    \begin{bmatrix} 0.21018744 \\ 0.31541596 \end{bmatrix} $$

$$ \textbf{Σ}_1 = \frac{1}{0.78523072} \cdot \left( \begin{align*}
        0.19258959 \cdot \begin{bmatrix} 0.19510580 & -0.11671576 \\ -0.11671576 & 0.06982144 \end{bmatrix} + \\
        0.23928490 \cdot \begin{bmatrix} 0.31169016 & -0.24328281 \\ -0.24328281 & 0.18988898 \end{bmatrix} + \\
        0.18443199 \cdot \begin{bmatrix} 0.00173954 &  0.00566236 \\  0.00566236 & 0.01843146 \end{bmatrix} + \\
        0.16892423 \cdot \begin{bmatrix} 0.05842267 & -0.11220983 \\ -0.11220983 & 0.21551643 \end{bmatrix} \end{align*}
    \right)
    =
    \begin{bmatrix} 0.15581144 & -0.12557173 \\ -0.12557173 & 0.12568249 \end{bmatrix} $$

$$ \textbf{Σ}_2 = \frac{1}{3.21476928} \cdot \left( \begin{align*}
        0.80741041 \cdot \begin{bmatrix} 0.15195383 & -0.08397185 \\ -0.08397185 & 0.04640404 \end{bmatrix} + \\
        0.76071510 \cdot \begin{bmatrix} 0.37232871 & -0.29568709 \\ -0.29568709 & 0.23482169 \end{bmatrix} + \\
        0.81556801 \cdot \begin{bmatrix} 0.00010378 & -0.00188044 \\ -0.00188044 & 0.03407127 \end{bmatrix} + \\
        0.83107577 \cdot \begin{bmatrix} 0.03602881 & -0.07885117 \\ -0.07885117 & 0.17257042 \end{bmatrix} \end{align*}
    \right)
    =
    \begin{bmatrix} 0.13560926 & -0.11192041 \\ -0.11192041 & 0.12047709 \end{bmatrix} $$

### 2) [2v]

**Given the new observation,**
$
\textbf{x}_{\text{new}} = \begin{bmatrix} 1 \\ 0.3 \\ 0.7 \end{bmatrix}
$
**, determine the cluster memberships (posteriors).**

In [ ]:
x_new = np.array([1, 0.3, 0.7])
π_new = np.array([π1_new, π2_new])
p_new = np.array([p1_new, p2_new])
μ_new = np.array([μ1_new, μ2_new])
Σ_new = np.array([Σ1_new, Σ2_new])

print("Gaussians:")
for k in range(1, 3):
	print("N(x_new | μ{}, Σ{}) = {:.8f}".format(k, k, gaussian(x_new[1:], μ_new[k-1], Σ_new[k-1])))
print("")

# Posteriors
for k in range(1, 3):
	print("P(c{} | x_new) = {:.8f}".format(k, (p_new[k-1] * gaussian(x_new[1:], μ_new[k-1], Σ_new[k-1]) * π_new[k-1]) / (p1_new * gaussian(x_new[1:], μ1_new, Σ1_new) * π1_new + p2_new * gaussian(x_new[1:], μ2_new, Σ2_new) * π2_new)))

Gaussians:
N(x_new | μ1, Σ1) = 0.03865732
N(x_new | μ2, Σ2) = 0.05926263

P(c1 | x_new) = 0.12581597
P(c2 | x_new) = 0.87418403


$$
P(c_k \; | \; \textbf{x}_{\text{new}})
    =
    \frac{p_k \cdot
    N \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} \; | \; \textbf{μ}_k, \textbf{Σ}_k \right)
    \cdot π_k}
    {\displaystyle\sum_{j = 1}^n \;
        p_j \cdot
        N \left( \begin{bmatrix} x_{i2} \\ x_{i3} \end{bmatrix} \; | \; \textbf{μ}_j, \textbf{Σ}_j \right)
        \cdot π_j}
$$

$$
p_1 \cdot
N \left( \begin{bmatrix} x_{\text{new},2} \\ x_{\text{new},3} \end{bmatrix} \; | \; \textbf{μ}_1, \textbf{Σ}_1 \right)
\cdot π_1
= 0.46039185 \cdot 0.03865732 \cdot 0.19630768 = 0.00349379
$$

$$
p_2 \cdot
N \left( \begin{bmatrix} x_{\text{new},2} \\ x_{\text{new},3} \end{bmatrix} \; | \; \textbf{μ}_2, \textbf{Σ}_2 \right)
\cdot π_2
= 0.50967458 \cdot 0.05926263 \cdot 0.80369232 = 0.02427525
$$

$$
P(c_1 \; | \; \textbf{x}_{\text{\text{new}}})
    = \frac{0.00349379}{0.00349379 + 0.02427525} = 0.12581597
$$

$$
P(c_2 \; | \; \textbf{x}_{\text{\text{new}}})
    = \frac{0.02427525}{0.00349379 + 0.02427525} = 0.87418403
$$


<!--
π1 = 0.19630768
π2 = 0.80369232

p1 = 0.46039185
p2 = 0.50967458

μ1 = [0.15829218 0.36423748]
μ2 = [0.21018744 0.31541596]

Σ1 = [[ 0.15581144 -0.12557173]
 [-0.12557173  0.12568249]]
Σ2 = [[ 0.13560926 -0.11192041]
 [-0.11192041  0.12047709]]

-->

### 3) [2.5v]

**Performing a hard assignment of observations to clusters under a ML assumption, identify the silhouette of the larger cluster under a Manhattan distance.**

### 4) [0.5v]

**Knowing the purity of the clustering solution is 0.75, identify the number of possible classes (ground truth).**

## II. Programming [9v]

**Recall the `column_diagnosis.arff` dataset from previous homeworks. For the following exercises, normalize the data using sklearn’s `MinMaxScaler`.**

In [ ]:
# Import Libraries

In [ ]:
# Data Loading

# Data Preprocessing

### 1) [4v]

**Using `sklearn`, apply k-means clustering fully unsupervisedly on the normalized data with**
$
k \in \{2, 3, 4, 5\}
$
**(random=0 and remaining parameters as default). Assess the silhouette and purity of the produced solutions.**

### 2) [2v]

**Consider the application of PCA after the data normalization:**

#### i.

**Identify the variability explained by the top two principal components.**

#### ii.

**For each one of these two components, sort the input variables by relevance by inspecting the absolute weights of the linear projection.**

### 3) [2v]

**Visualize side-by-side the data using: i) the ground diagnoses, and ii) the previously learned**
$
k = 3
$
**clustering solution. To this end, projected the normalized data onto a 2-dimensional data space using PCA and then color observations using the reference and cluster annotations.**

### 4) [1v]

**Considering the results from questions (1) and (3), identify two ways on how clustering can be used to characterize the population of ill and healthy individuals.**